In [47]:
import pickle
import pandas as pd
import numpy as np

In [48]:
actual_test_data=pd.read_csv('Data/Assignment_Test.csv')

In [49]:
x_test=pd.read_csv("cleaned_dataset/cleaned_test_data.csv")
x_test.head()

,DEALER ID,Cibil Score,TOTAL ASSET COST,ASSET CTG,ASSET MODEL NO,MARITAL STATUS,GENDER,ADDRESS TYPE,EMPLOY CONSTITUTION,EMPLOYER TYPE,phone_digitalage
0,105615,757.0,95352.0,SCECA,143241,Married,Male,Self/Spouse Owned,SALARIED,SALARIED,676
1,91593,750.0,75371.0,MOECA,166536,Married,Male,Parental,SELF-EMPLOYED,SELF-EMPLOYED,897
2,74152,710.0,94983.0,SCECA,134123,Single,Male,Self/Spouse Owned,SELF-EMPLOYED,SELF-EMPLOYED,839
3,110164,695.0,94535.5,MCECC,143627,Married,Male,RESIDENCE,SALARIED,Government,2050
4,113037,726.0,90277.0,MCECA,143948,Single,Male,Parental,SELF-EMPLOYED,SELF-EMPLOYED,-1


In [50]:
with open('models/Random.pkl', 'rb') as f:
    Random = pickle.load(f)

with open('models/Label_encoder.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

There is new Category in ADDRESS TYPE  column 'OTHERS' replaying it with -1

In [51]:
categorical_cols = x_test.select_dtypes(include='object').columns

# Transform each categorical column in the test set
for col in categorical_cols:
    if col in label_encoders:
        le = label_encoders[col]  # Get the corresponding LabelEncoder
        current_classes = le.classes_
        
        # Replace unseen labels with -1
        x_test[col] = x_test[col].apply(lambda x: x if x in current_classes else -1)
        
        # Transform the data
        x_test[col] = x_test[col].apply(lambda x: le.transform([x])[0] if x != -1 else -1)
    else:
        print(f"Warning: Column {col} not found in the training set encoders.")

In [52]:
x_test.isnull().sum()

DEALER ID              0
Cibil Score            0
TOTAL ASSET COST       0
ASSET CTG              0
ASSET MODEL NO         0
MARITAL STATUS         0
GENDER                 0
ADDRESS TYPE           0
EMPLOY CONSTITUTION    0
EMPLOYER TYPE          0
phone_digitalage       0
dtype: int64

In [53]:
pred2=Random.predict(x_test)
df1 = pd.DataFrame({'Application Status': pred2})
df1['Application Status'] = df1['Application Status'].map({1: 'APPROVED', 0: 'DECLINED'})
df1.value_counts()


Application Status
APPROVED              1636
DECLINED               364
Name: count, dtype: int64

In [54]:
df_final =pd.DataFrame({'UID': actual_test_data['UID'], 'Prediction': df1['Application Status']})


In [56]:
df_final.to_csv("Test Predictions/predictions.csv",index=False)
